# Langgraph + ReAct Agent + Mathematical tools
## Please note that this notebook uses the latest way to create and use ReAct agents and OpenAI models using Langchain

### PLease visit https://docs.langchain.com/oss/python/langchain/agents and https://docs.langchain.com/oss/python/integrations/chat/openai for more details

In [38]:
import os
from typing import Dict, Any
import re
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [4]:
openai_api_key = "<API_KEY>"
os.environ["OPENAI_API_KEY"] = openai_api_key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [6]:
@tool
def plus(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtract b from a."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def divide(a: float, b: float) -> str:
    """Divide a by b; handles division by zero."""
    if b == 0:
        return "Error: Division by zero is not allowed."
    return a / b


TOOLS = [plus, subtract, multiply, divide]

In [83]:
from langchain_core.callbacks import BaseCallbackHandler


class VerboseToolLogger(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, run_id=None, **kwargs):
        print(f"🔧 Tool starts | input: {input_str}")

    def on_tool_end(self, output, run_id=None, **kwargs):
        print(f"✅ Tool ends | output: {output}")

    def on_llm_start(self, serialized, prompts, run_id=None, **kwargs):
        print(f"🧠 LLM start | prompts: {prompts}")

    def on_llm_end(self, response, run_id=None, **kwargs):
        print(f"🧠 LLM end | response: {response}")

logger_handler = VerboseToolLogger() 

In [52]:
MATH_PATTERN = re.compile(
    r"(\d+)\s*(plus|add|minus|subtract|times|multiply|divided by|divide)\s*(\d+)",
    re.IGNORECASE
)

def is_math_query(text: str) -> bool:
    """Return True if the user query contains basic math operations."""
    return bool(MATH_PATTERN.search(text))

In [54]:
class AgentState(Dict):
    user_query: str
    result: Any


In [66]:
def build_math_agent():
    model = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY"),
    )

    tools = [plus, subtract, multiply, divide]

    agent = create_agent(
        model=model,
        tools=tools,
        system_prompt=(
            "You are a math assistant. "
            "Given a math problem, decide the correct tool to use. "
            "Only use the tools for calculations."
        ),
    )

    return agent


def math_node(state: AgentState):
    """Handles a math query using an LLM agent that calls tools automatically."""
    query = state["user_query"]

    match = MATH_PATTERN.search(query)

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    if not match:
        classification = llm.invoke([
            SystemMessage(content=(
                "Classify whether this query is a basic arithmetic question. "
                "Respond ONLY with 'math' or 'not math'."
            )),
            HumanMessage(content=query)
        ]).content.strip().lower()

        if classification != "math":
            return {"result": None}

        extract = llm.invoke([
            SystemMessage(content="Extract arithmetic expressions into structured JSON."),
            HumanMessage(content=f"""
Extract the math operation from the following query:
"{query}"

Return only JSON: 
{{ "instruction": "<rewrite into a solvable arithmetic description>" }}
            """)
        ]).content

        import json
        try:
            instruction_json = json.loads(extract)
            rewritten_query = instruction_json["instruction"]
        except:
            rewritten_query = query
    else:
        a = match.group(1)
        op = match.group(2)
        b = match.group(3)
        rewritten_query = f"Compute: {a} {op} {b}"

    math_agent = build_math_agent()
    result = math_agent.invoke({
        "messages": [
            {"role": "user", "content": rewritten_query}
        ]
    })

    final_ai_msg = [
        m for m in result["messages"]
        if isinstance(m, AIMessage)
    ][-1]

    return {"result": final_ai_msg.content}


In [68]:
def general_node(state: AgentState):
    """LLM answers general knowledge or conversational questions."""
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.2,
        api_key=os.getenv("OPENAI_API_KEY"),
    )
    response = llm.invoke(
        [
            SystemMessage(content="You are a helpful general assistant."),
            HumanMessage(content=state["user_query"])
        ]
    )
    return {"result": response.content}


In [70]:
def route_to_math(state: AgentState):
    return is_math_query(state["user_query"])


def create_agent_graph():
    graph = StateGraph(AgentState)

    graph.add_node("math_node", math_node)
    graph.add_node("general_node", general_node)

    graph.set_entry_point("general_node")

    graph.add_conditional_edges(
        "general_node",
        route_to_math,
        {
            True: "math_node",
            False: END
        }
    )

    graph.add_edge("math_node", END)

    return graph.compile()

### Tool calling is logged in the process to ensure that LLM is nto generating response with its own reasoning.

In [93]:
agent = create_agent_graph()


In [95]:
print(agent.invoke({"user_query": "What is 5 plus 3?"},config={"callbacks": [logger_handler]}))

🧠 LLM start | prompts: ['System: You are a helpful general assistant.\nHuman: What is 5 plus 3?']
🧠 LLM end | response: generations=[[ChatGeneration(text='5 plus 3 equals 8.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='5 plus 3 equals 8.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 26, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Cf0ZDQFTxqDexo1dmCEoWZ3N6tb8r', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--93111ae3-ce72-4b37-85e5-40b10c7215c4-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34, 'input_token_deta

Note: Tool is being used. However, its involving multiple steps unnecessarily. This is because,
1. First a general_node is invoked which tries to classify wheather it requires a mathematical direction or general direction.
2. Then the tool node with the ReAct agent
3. Then a final summarizer agent, which makes it complex. 


In [99]:
print(agent.invoke({"user_query": "Tell me something about space travel."},config={"callbacks": [logger_handler]}))

🧠 LLM start | prompts: ['System: You are a helpful general assistant.\nHuman: Tell me something about space travel.']
🧠 LLM end | response: generations=[[ChatGeneration(text="Space travel refers to the act of traveling beyond Earth's atmosphere, typically involving spacecraft designed for human or robotic exploration of outer space. Here are some key points about space travel:\n\n1. **History**: The era of human space travel began with the launch of Sputnik 1 by the Soviet Union in 1957, followed by Yuri Gagarin becoming the first human in space in 1961. The United States' Apollo program culminated in the historic Apollo 11 mission, where Neil Armstrong and Buzz Aldrin landed on the Moon in 1969.\n\n2. **Types of Spacecraft**: Space travel can involve various types of spacecraft, including crewed spacecraft (like the Space Shuttle, Soyuz, and Crew Dragon) and uncrewed probes (like Voyager and Mars rovers). Spacecraft are designed to withstand the harsh conditions of space, including va

### Drawback: It cannot handle multiple tool calls within a mathematical operation right now

In [101]:
print(agent.invoke(
    {
        "user_query": """
1) What is 5 plus 3?
2) What is 10 minus 4?
3) What is 6 times 7?
4) What is 20 divided by 5?
5) Tell me something about space travel.
6) Do numbers have meaning beyond mathematics?
        """
    },
    config={"callbacks": [logger_handler]}
))

🧠 LLM start | prompts: ['System: You are a helpful general assistant.\nHuman: \n1) What is 5 plus 3?\n2) What is 10 minus 4?\n3) What is 6 times 7?\n4) What is 20 divided by 5?\n5) Tell me something about space travel.\n6) Do numbers have meaning beyond mathematics?\n        ']
🧠 LLM end | response: generations=[[ChatGeneration(text='1) 5 plus 3 equals 8.\n2) 10 minus 4 equals 6.\n3) 6 times 7 equals 42.\n4) 20 divided by 5 equals 4.\n5) Space travel involves the exploration of outer space through the use of spacecraft. It has advanced significantly since the first human spaceflight in 1961, with missions to the Moon, Mars exploration, and the establishment of the International Space Station (ISS), which serves as a microgravity research laboratory.\n6) Yes, numbers can have meanings beyond mathematics. In various cultures, certain numbers are associated with symbolism or significance. For example, the number 7 is often considered lucky in many cultures, while the number 13 is viewed a

#### At least for this usecase - multiple steps isn't necessary - but it helps learn the way tools work when we see every steps in the logs